In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report

import random
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If you are using CUDA
torch.backends.cudnn.deterministic = True  # For deterministic results
torch.backends.cudnn.benchmark = False  # For consistency across different environments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = 'D:\\PAD-UFES\\images'  
METADATA_PATH = 'D:\\PAD-UFES\\metadata.csv'

metadata = pd.read_csv(METADATA_PATH)

def preprocess_metadata(metadata):
    metadata = metadata.fillna('UNK')

    boolean_cols = [
        'smoke',
        'drink',
        'pesticide',
        'skin_cancer_history',
        'cancer_history',
        'has_piped_water',
        'has_sewage_system',
        'itch',
        'grew',
        'hurt',
        'changed',
        'bleed',
        'elevation',
        'biopsed',
    ]
    # Ensure columns are strings and lowercase
    for col in boolean_cols:
        metadata[col] = metadata[col].astype(str).str.lower()
    
    # Map boolean columns to 1/0/-1
    boolean_mapping = {'true': 1, 'false': 0, 'unk': -1}
    for col in boolean_cols:
        metadata[col] = metadata[col].map(boolean_mapping)
    
    # Handle categorical variables
    categorical_cols = [
        'background_father',
        'background_mother',
        'gender',
        'region',
        'diagnostic',
    ]
    # Convert categorical columns to string
    for col in categorical_cols:
        metadata[col] = metadata[col].astype(str)
    
    # One-hot encode categorical variables
    metadata_encoded = pd.get_dummies(metadata[categorical_cols])
    
    # Normalize numerical variables
    numerical_cols = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']
    # Ensure numerical columns are numeric
    for col in numerical_cols:
        metadata[col] = pd.to_numeric(metadata[col], errors='coerce')
    # Fill NaNs in numerical columns with the mean
    metadata[numerical_cols] = metadata[numerical_cols].fillna(metadata[numerical_cols].mean())
    # Scale numerical columns
    scaler = StandardScaler()
    metadata_numeric = metadata[numerical_cols]
    metadata_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(metadata_numeric), columns=numerical_cols
    )
    
    # Combine all metadata features
    metadata_processed = pd.concat(
        [metadata_numeric_scaled.reset_index(drop=True),
         metadata_encoded.reset_index(drop=True),
         metadata[boolean_cols].reset_index(drop=True)], axis=1
    )
    
    return metadata_processed

# Preprocess metadata
metadata_processed = preprocess_metadata(metadata)

def get_image_paths(metadata, image_dir):
    image_paths = []
    for idx, row in metadata.iterrows():
        filename = row['img_id']
        # Ensure filename is a string
        filename = str(filename)
        # Check if filename has an extension
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            # Try common extensions
            possible_extensions = ['.jpg', '.jpeg', '.png']
            found = False
            for ext in possible_extensions:
                filepath = os.path.join(image_dir, filename + ext)
                if os.path.isfile(filepath):
                    image_paths.append(filepath)
                    found = True
                    break
            if not found:
                print(f"Image file not found for ID: {filename}")
                image_paths.append(None)
        else:
            filepath = os.path.join(image_dir, filename)
            if os.path.isfile(filepath):
                image_paths.append(filepath)
            else:
                print(f"Image file not found: {filepath}")
                image_paths.append(None)
    metadata['ImagePath'] = image_paths
    return metadata

metadata = get_image_paths(metadata, IMAGE_DIR)

# Remove entries with missing images
metadata = metadata[metadata['ImagePath'].notnull()]
metadata_processed = metadata_processed.loc[metadata.index].reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

# Drop diagnostic-related columns from features
diagnostic_cols = ['diagnostic_ACK', 'diagnostic_BCC', 'diagnostic_MEL', 'diagnostic_NEV', 'diagnostic_SCC', 'diagnostic_SEK']
metadata_processed = metadata_processed.drop(columns=diagnostic_cols)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(metadata['diagnostic'])
num_classes = len(label_encoder.classes_)

# Split data into features and labels
X_meta = metadata_processed.reset_index(drop=True)
X_img_paths = metadata['ImagePath'].reset_index(drop=True)
y = pd.Series(y_encoded)

X_train_meta, X_temp_meta, X_train_img_paths, X_temp_img_paths, y_train, y_temp = train_test_split(
    X_meta,
    X_img_paths,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_val_meta, X_test_meta, X_val_img_paths, X_test_img_paths, y_val, y_test = train_test_split(
    X_temp_meta,
    X_temp_img_paths,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# Load augmented metadata + image paths
aug_meta_df   = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
aug_labels_df = pd.read_csv("D:/PAD-UFES/augmented_labels.csv")

# Combine augmented samples with training set
X_train_meta_final = pd.concat([X_train_meta, aug_meta_df], ignore_index=True)
X_train_img_paths_final = pd.concat([X_train_img_paths.reset_index(drop=True),
                                     aug_labels_df['ImagePath']], ignore_index=True)
y_train_final = pd.concat([y_train.reset_index(drop=True),
                           aug_labels_df['Label']], ignore_index=True)

class PADUFESDataset(Dataset):
    def __init__(self, img_paths, meta_data, labels, transform=None):
        self.img_paths = img_paths.reset_index(drop=True)
        self.meta_data = meta_data.reset_index(drop=True)
        self.labels = pd.Series(labels).reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        meta = torch.tensor(self.meta_data.iloc[idx].values.astype(np.float32))
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return image, meta, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomRotation(70),          
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = PADUFESDataset(X_train_img_paths_final, X_train_meta_final, y_train_final, transform=train_transform)
val_dataset = PADUFESDataset(X_val_img_paths, X_val_meta, y_val, transform=val_test_transform)
test_dataset = PADUFESDataset(X_test_img_paths, X_test_meta, y_test, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"\n✅ Loaded Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
# Decode class labels
train_labels_decoded = label_encoder.inverse_transform(y_train_final)
val_labels_decoded = label_encoder.inverse_transform(y_val)
test_labels_decoded = label_encoder.inverse_transform(y_test)

# Count class instances
train_class_counts = pd.Series(train_labels_decoded).value_counts()
val_class_counts = pd.Series(val_labels_decoded).value_counts()
test_class_counts = pd.Series(test_labels_decoded).value_counts()

# Print distributions
print("\n📊 Class distribution in Augmented Training Set:")
print(train_class_counts)

print("\n📊 Class distribution in Validation Set:")
print(val_class_counts)

print("\n📊 Class distribution in Test Set:")
print(test_class_counts)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
optimizers = ['Adam', 'SGD', 'RMSprop', 'Adagrad']
val_accuracy = [87.35, 68.97, 85.38, 78.26]
val_f1 = [87.27, 68.24, 85.40, 77.90]

# Bar positions
x = np.arange(len(optimizers))
bar_width = 0.35

# Viridis color palette for all bars (2 bars per optimizer)
cmap = plt.cm.viridis
colors = [cmap(i / (len(optimizers) * 2)) for i in range(len(optimizers) * 2)]

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

# Draw bars with unique colors
bars1 = ax.bar(x - bar_width/2, val_accuracy, width=bar_width, label='Val Accuracy', color=colors[0::2])
bars2 = ax.bar(x + bar_width/2, val_f1, width=bar_width, label='Val F1 Score', color=colors[1::2])

# Axes and title
ax.set_xlabel('Optimizers', fontweight='bold')
ax.set_ylabel('Validation Score (%)', fontweight='bold')
ax.set_title('Optimizer Comparison: Validation Accuracy vs F1 Score', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(optimizers, fontweight='bold')
ax.tick_params(axis='y', labelsize=10)
ax.tick_params(axis='x', labelsize=10)
# ax.legend()

from matplotlib.patches import Patch

custom_legend = [
    Patch(color=colors[0], label='Adam - Acc'),
    Patch(color=colors[1], label='Adam - F1'),
    Patch(color=colors[2], label='SGD - Acc'),
    Patch(color=colors[3], label='SGD - F1'),
    Patch(color=colors[4], label='RMSprop - Acc'),
    Patch(color=colors[5], label='RMSprop - F1'),
    Patch(color=colors[6], label='Adagrad - Acc'),
    Patch(color=colors[7], label='Adagrad - F1')
]
ax.legend(handles=custom_legend, bbox_to_anchor=(1.05, 1), loc='upper left')

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9, fontweight='bold')

# Layout
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# Data
batch_sizes = ['16', '32', '64']
val_accuracy = [85.38, 87.15, 85.77]
val_f1 = [85.31, 87.10, 85.46]

# Bar positions
x = np.arange(len(batch_sizes))
bar_width = 0.35

# Use magma colormap for 6 unique bars
cmap = plt.cm.magma
colors = [cmap(i / 6) for i in range(6)]

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - bar_width/2, val_accuracy, width=bar_width, label='Val Accuracy', color=colors[0::2])
bars2 = ax.bar(x + bar_width/2, val_f1, width=bar_width, label='Val F1 Score', color=colors[1::2])

# Axis formatting
ax.set_xlabel('Batch Size', fontweight='bold')
ax.set_ylabel('Validation Score (%)', fontweight='bold')
ax.set_title('Batch Size vs Validation Accuracy and F1 Score', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(batch_sizes, fontweight='bold')
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Custom Legend
custom_legend = [
    Patch(color=colors[0], label='Batch Size 16 - Acc'),
    Patch(color=colors[1], label='Batch Size 16 - F1'),
    Patch(color=colors[2], label='Batch Size 32 - Acc'),
    Patch(color=colors[3], label='Batch Size 32 - F1'),
    Patch(color=colors[4], label='Batch Size 64 - Acc'),
    Patch(color=colors[5], label='Batch Size 64 - F1'),
]
ax.legend(handles=custom_legend, bbox_to_anchor=(1.05, 1), loc='upper left')

# Annotate exact values
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9, fontweight='bold')

# Layout
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data
temperature = [1, 3, 6, 9]
accuracy = [0.875494, 0.859684, 0.865613, 0.877470]
f1_score = [0.874773, 0.859024, 0.864872, 0.876967]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(temperature, accuracy, marker='o', label='Validation Accuracy', linewidth=4)
plt.plot(temperature, f1_score, marker='s', label='Validation F1 Score', linewidth=4)

# Axis and Title Formatting
plt.xlabel('Temperature', fontweight='bold')
plt.ylabel('Validation Score (%)', fontweight='bold')
plt.title('Effect of Temperature on Accuracy and F1 Score', fontweight='bold')

# Axis Ticks and Grid
plt.xticks(temperature, fontweight='bold')
plt.yticks(fontweight='bold')

# Legend
plt.legend()

# Optional: Annotate data points
for t, acc, f1 in zip(temperature, accuracy, f1_score):
    plt.text(t, acc + 0.001, f'{acc:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    plt.text(t, f1 - 0.002, f'{f1:.3f}', ha='center', va='top', fontsize=9, fontweight='bold')

# Optional: Annotate data points
for t, acc, f1 in zip(temperature, accuracy, f1_score):
    plt.text(t, acc + 0.001, f'{acc:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    plt.text(t, f1 - 0.002, f'{f1:.3f}', ha='center', va='top', fontsize=9, fontweight='bold')

# Adjust y-axis limits to keep annotations visible
min_y = min(min(accuracy), min(f1_score)) - 0.005
max_y = max(max(accuracy), max(f1_score)) + 0.005
plt.ylim(min_y, max_y)

# Layout and show
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data
alpha = [0.2, 0.4, 0.6, 0.8]
accuracy = [0.871542, 0.867589, 0.865613, 0.867589]
f1_score = [0.869651, 0.868283, 0.865297, 0.865305]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(alpha, accuracy, marker='o', label='Val Accuracy', linewidth=3)
plt.plot(alpha, f1_score, marker='s', label='Val F1 Score', linewidth=3)

# Connect Accuracy and F1 points with lines per alpha
for a, acc, f1 in zip(alpha, accuracy, f1_score):
    plt.plot([a, a], [acc, f1], color='gray', linestyle='--', linewidth=1.5)

# Annotate values
for a, acc, f1 in zip(alpha, accuracy, f1_score):
    plt.text(a, acc + 0.001, f'{acc:.3f}', ha='center', fontsize=9, fontweight='bold')
    plt.text(a, f1 - 0.002, f'{f1:.3f}', ha='center', fontsize=9, fontweight='bold')

# Axis formatting
plt.xlabel('Alpha', fontweight='bold')
plt.ylabel('Validation Score', fontweight='bold')
plt.title('Accuracy vs F1 Score for Different Alpha Values', fontweight='bold')
plt.xticks(alpha, fontweight='bold')
plt.yticks(fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data
alpha = [0.2, 0.4, 0.6, 0.8]
accuracy = [0.871542, 0.867589, 0.865613, 0.867589]
f1_score = [0.869651, 0.868283, 0.865297, 0.865305]

# Plot
plt.figure(figsize=(10, 6))

# Plot lines
plt.plot(alpha, accuracy, marker='o', label='Val Accuracy', linewidth=3, color='purple')
plt.plot(alpha, f1_score, marker='s', label='Val F1 Score', linewidth=3, color='magenta')

# Fill area between the two lines
plt.fill_between(alpha, accuracy, f1_score, color='violet', alpha=0.3, label='Difference Area')

# Annotate points
for a, acc, f1 in zip(alpha, accuracy, f1_score):
    plt.text(a, acc + 0.001, f'{acc:.3f}', ha='center', fontsize=9, fontweight='bold')
    plt.text(a, f1 - 0.002, f'{f1:.3f}', ha='center', fontsize=9, fontweight='bold')

# Axis formatting
plt.xlabel('Alpha', fontweight='bold')
plt.ylabel('Validation Score', fontweight='bold')
plt.title('Validation Accuracy vs F1 Score with Alpha', fontweight='bold')
plt.xticks(alpha, fontweight='bold')
plt.yticks(fontweight='bold')
plt.legend()

# Adjust y-axis limits to keep annotations visible
min_y = min(min(accuracy), min(f1_score)) - 0.005
max_y = max(max(accuracy), max(f1_score)) + 0.005
plt.ylim(min_y, max_y)

# Layout and show
plt.tight_layout()
plt.show()

In [ ]:
# Convert encoded labels back to original class names
y_train_labels = label_encoder.inverse_transform(y_train)
y_val_labels = label_encoder.inverse_transform(y_val)
y_test_labels = label_encoder.inverse_transform(y_test)

# Count instances of each class
train_class_counts = pd.Series(y_train_labels).value_counts()
val_class_counts = pd.Series(y_val_labels).value_counts()
test_class_counts = pd.Series(y_test_labels).value_counts()

# Print counts
print("Class distribution in Training Set:")
print(train_class_counts)

print("\nClass distribution in Validation Set:")
print(val_class_counts)

print("\nClass distribution in Test Set:")
print(test_class_counts)


In [ ]:
metadata_processed.info()

In [ ]:
print(num_classes)

In [ ]:
import torch
import torch.nn as nn
import timm  
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class EarlyFusionModel(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 56 * 56),  # Smaller initial dimension
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )
        
        # Load the InceptionResNetV2 model from timm
        self.inception_resnet = timm.create_model('inception_resnet_v2', pretrained=True)
        
        # Modify the first convolutional block (conv2d_1a) to accept 4 channels
        first_conv = self.inception_resnet.conv2d_1a.conv
        self.inception_resnet.conv2d_1a.conv = nn.Conv2d(
            4, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        )
        
        # Copy weights for the first 3 channels and initialize the 4th channel
        with torch.no_grad():
            self.inception_resnet.conv2d_1a.conv.weight[:, :3] = first_conv.weight
            # Initialize the new channel with the mean of existing weights
            self.inception_resnet.conv2d_1a.conv.weight[:, 3:] = first_conv.weight.mean(dim=1, keepdim=True) * 0.1
        
        # Modify the final classification head to match the number of classes
        in_features = self.inception_resnet.classif.in_features
        self.inception_resnet.classif = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, num_classes)
        )
        
    def forward(self, img, meta):
        # Reshape metadata to image-like format
        batch_size = img.shape[0]
        meta_reshaped = self.meta_embed(meta).view(batch_size, 1, 56, 56)
        
        # Upsample metadata to match image dimensions
        meta_upsampled = F.interpolate(meta_reshaped, 
                                       size=(224, 224), 
                                       mode='bilinear', 
                                       align_corners=False)
        
        # Concatenate image and metadata along the channel dimension
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Process through the modified InceptionResNetV2
        out = self.inception_resnet(combined_input)
        return out
        
input_dim_meta = X_train_meta.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EarlyFusionModel(input_dim_meta, num_classes).to(device)

from torchinfo import summary
summary(model=model, 
        input_size=[(16, 3, 224, 224), (16, input_dim_meta)],  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
print(input_dim_meta)

In [ ]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_accuracy_max = -np.Inf
        
    def __call__(self, val_acc, model):
        score = val_acc
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation accuracy increased ({self.val_accuracy_max:.6f} --> {val_acc:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_accuracy_max = val_acc

from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

true_labels, pred_labels = test(model, test_loader, device)

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, meta, labels in pbar:
        images, meta, labels = images.to(device), meta.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, meta)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Optional: Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{running_loss/total:.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(train_loader), correct/total

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, meta, labels in val_loader:
            images, meta, labels = images.to(device), meta.to(device), labels.to(device)
            
            outputs = model(images, meta)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(val_loader), correct/total

def train_model_with_scheduler_and_checkpoint(
    model, train_loader, val_loader, optimizer, criterion, device, 
    epochs=20, patience=5, scheduler_patience=5, checkpoint_dir='checkpoints'):
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'inceptionresnetv2.pt')
    
    early_stopping = EarlyStopping(
        patience=patience, 
        verbose=True, 
        path=checkpoint_path
    )
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max',  # Changed to max since we're monitoring accuracy
        patience=scheduler_patience, 
        verbose=True,
        factor=0.1,
        min_lr=1e-6
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_model_epoch = None
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        
        # Training phase
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Validation phase
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Update scheduler based on validation accuracy
        scheduler.step(val_acc)
        
        # Early stopping check
        early_stopping(val_acc, model)
        if val_acc > early_stopping.val_accuracy_max:
            best_model_epoch = epoch + 1
            
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    
    # Plot training curves
    plot_training_curves_with_checkpoint(history, best_model_epoch)
    
    return model, history

def plot_training_curves_with_checkpoint(history, best_model_epoch):
    epochs_range = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    # Loss curves
    ax1.plot(epochs_range, history['train_loss'], label='Training Loss')
    ax1.plot(epochs_range, history['val_loss'], label='Validation Loss')
    if best_model_epoch:
        ax1.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    
    # Accuracy curves
    ax2.plot(epochs_range, history['train_acc'], label='Training Accuracy')
    ax2.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    if best_model_epoch:
        ax2.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    
    # Learning rate curve
    ax3.plot(epochs_range, history['lr'], label='Learning Rate')
    if best_model_epoch:
        ax3.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_title('Learning Rate Schedule')
    ax3.set_yscale('log')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Function to evaluate test metrics
def evaluate_test_metrics(model, test_loader, device):
    true_labels, pred_labels = test(model, test_loader, device)
    acc = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='macro')
    recall = recall_score(true_labels, pred_labels, average='macro')
    f1 = f1_score(true_labels, pred_labels, average='macro')
    return acc, precision, recall, f1

# Placeholder for results
results = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1_score": []
}

best_accuracy = 0.0
best_model_state = None

# Run experiment for 3 random seeds
seeds = [42, 123, 569]  # Example random seeds
for seed in seeds:
    print(f"\nTraining with random seed: {seed}")
    set_random_seed(seed)
    
    # Reinitialize model, optimizer, and criterion
    model = EarlyFusionModel(input_dim_meta, num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    
    # Train the model
    model, history = train_model_with_scheduler_and_checkpoint(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=100,
        patience=7,
        scheduler_patience=3,
        checkpoint_dir='D:\\PAD-UFES\\checkpoints'
    )
    
    # Evaluate on test set
    acc, precision, recall, f1 = evaluate_test_metrics(model, test_loader, device)
    print(f"Seed {seed}: Accuracy={acc:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1 Score={f1:.4f}")
    
    # Save metrics
    results["accuracy"].append(acc)
    results["precision"].append(precision)
    results["recall"].append(recall)
    results["f1_score"].append(f1)
    
    # Update the best model
    if acc > best_accuracy:
        best_accuracy = acc
        best_model_state = model.state_dict()

# Compute average and standard deviation
metrics_summary = {}
for metric, values in results.items():
    avg = np.mean(values)
    std_dev = np.std(values)
    metrics_summary[metric] = (avg, std_dev)
    print(f"{metric.capitalize()}: Mean={avg:.4f}, StdDev={std_dev:.4f}")

# Save the best model
print(f"Best model achieved an accuracy of {best_accuracy:.4f}")
torch.save(best_model_state, 'D:\\PAD-UFES\\best_early_fusion_inceptionresnetv2smoteDA.pth')

model = EarlyFusionModel(input_dim_meta, num_classes).to(device)
model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_inceptionresnetv2smoteDA.pth'))
model.eval()
print("Best model loaded successfully!")

In [ ]:
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

true_labels, pred_labels = test(model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
fpr = dict()
tpr = dict()
roc_auc = dict()

n_class = 6

for i in range(n_class):
    fpr[i], tpr[i], _ = roc_curve(np.array(true_labels) == i, np.array(pred_labels) == i)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
colors = ['orange', 'green', 'red', 'blue', 'purple', 'pink']

for i in range(n_class):
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=2, label=f'ROC curve (AUC = {roc_auc[i]:.2f}) for {class_names[i]}')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('(ROC)Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()